In [1]:
import numpy as np 
import pandas as pd
from sklearn import neighbors
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [3]:
# read train csv
prop_1_df = pd.read_csv("./Resources/properties_2016.csv")
prop_2_df = pd.read_csv("./Resources/properties_2017.csv")
prop_list = [prop_1_df, prop_2_df]

In [4]:
for i in prop_list:
    i.loc[:,'latitude'] = i.loc[:,'latitude']/1e6
    i.loc[:,'longitude'] = i.loc[:,'longitude']/1e6

In [5]:
prop_un_1_df = prop_1_df.loc[prop_1_df['latitude'].isna(),:]
prop_un_2_df = prop_2_df.loc[prop_2_df['latitude'].isna(),:]
prop_1_df = prop_1_df.loc[prop_1_df['latitude'].notna(),:]
prop_2_df = prop_2_df.loc[prop_2_df['latitude'].notna(),:]

In [6]:
from sklearn import neighbors
from sklearn.preprocessing import OneHotEncoder

def zoningcode2int( df, target ):
    storenull = df[ target ].isnull()
    enc = LabelEncoder( )
    df[ target ] = df[ target ].astype( str )

    print('fit and transform')
    df[ target ]= enc.fit_transform( df[ target ].values )
    print( 'num of categories: ', enc.classes_.shape  )
    df.loc[ storenull, target ] = np.nan
    print('recover the nan value')
    return enc

def fillna_knn( df, base, target, fraction = 0.1, threshold = 10, n_neighbors = 5 ):
    assert isinstance( base , list ) or isinstance( base , np.ndarray ) and isinstance( target, str ) 
    whole = [ target ] + base
    
    miss = df[target].isnull()
    notmiss = ~miss 
    nummiss = miss.sum()
    
    enc = OneHotEncoder()
    X_target = df.loc[ notmiss, whole ].sample( frac = fraction )
    
    enc.fit( X_target[ target ].unique().reshape( (-1,1) ) )
    
    Y = enc.transform( X_target[ target ].values.reshape((-1,1)) ).toarray()
    X = X_target[ base  ]
    
    print( 'fitting' )
    n_neighbors = n_neighbors
    clf = neighbors.KNeighborsClassifier( n_neighbors, weights = 'uniform' )
    clf.fit( X, Y )
    
    print( 'the shape of active features: ' ,enc.active_features_.shape )
    
    print( 'predicting' )
    Z = clf.predict(df.loc[miss, base])
    
    numunperdicted = Z[:,0].sum()
    if numunperdicted / nummiss *100 < threshold :
        print( 'writing result to df' )    
        df.loc[ miss, target ]  = np.dot( Z , enc.active_features_ )
        print( 'num of unperdictable data: ', numunperdicted )
        return enc
    else:
        print( 'out of threshold: {}% > {}%'.format( numunperdicted / nummiss *100 , threshold ) )

In [8]:
geo_col = ['buildingqualitytypeid', 'regionidcity', 'regionidneighborhood', 
           'regionidzip', 'unitcnt', 'yearbuilt', 'lotsizesquarefeet']
geo_trans = ['propertycountylandusecode', 'propertyzoningdesc']

In [12]:
prop_1_df.propertyzoningdesc.isna().sum()

995151

In [7]:
import warnings
warnings.filterwarnings('ignore')
zoningcode2int( df = prop_1_df, target = 'propertyzoningdesc')
fillna_knn(df = prop_1_df, base = ['latitude', 'longitude'], target = 'propertyzoningdesc', fraction = 0.15)
zoningcode2int( df = prop_2_df, target = 'propertyzoningdesc')
fillna_knn(df = prop_2_df, base = ['latitude', 'longitude'], target = 'propertyzoningdesc', fraction = 0.15)  

fit and transform
num of categories:  (5639,)
recover the nan value
fitting
the shape of active features:  (3292,)
predicting


MemoryError: Unable to allocate array with shape (995151, 3292) and data type float64

In [ ]:
prop_1_df = pd.concat([prop_1_df, prop_un_1_df])
prop_2_df = pd.concat([prop_2_df, prop_un_2_df])
prop_1_df[['parcelid', 'propertyzoningdesc']].to_csv("./Resources/properties_propertyzoningdesc_2016.csv")
prop_2_df[['parcelid', 'propertyzoningdesc']].to_csv("./Resources/properties_propertyzoningdesc_2017.csv")